In [48]:
# Importing the libraries

import numpy as np 
import pandas as pd
from tqdm import tqdm
import math
import matplotlib.pylab as plt
from scipy import sparse
from scipy.linalg import sqrtm
import pandas as pd
import numpy as np
from zipfile import ZipFile
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from pathlib import Path
import matplotlib.pyplot as plt
from tqdm import tqdm

In [49]:
complete_train = pd.read_csv('train.csv')
complete_train_og = complete_train.copy()
test = pd.read_csv('kaggle_baseline.csv')

In [50]:
df = complete_train
def set_top_scores(group):
    # Find the highest rating
    highest_rating = group['rating'].max()
    if highest_rating == 5 or highest_rating == 4:
        group['new_rating'] = group['rating'].apply(lambda x: 1 if x == highest_rating or x == highest_rating - 1 else 0)
    else:
        group['new_rating'] = group['rating'].apply(lambda x: 1 if x == highest_rating else 0)        
    
    return group

def only_the_best(group):
    # Find the highest rating
    highest_rating = group['rating'].max()
    group['new_rating'] = group['rating'].apply(lambda x: 1 if x == 5 or x == 4 else 0) 
    
    return group

df = df.groupby('user_id').apply(only_the_best).reset_index(drop=True)
df = df.drop(['rating'], axis=1)  # Remove original rating column
df.rename(columns={'new_rating': 'rating'}, inplace=True)
df = df.drop(['age', 'release_date', 'sex'], axis=1)
df

/var/folders/yb/mm0hrmlx3g9dv10c735753m00000gn/T/ipykernel_15887/2104233964.py:19: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  df = df.groupby('user_id').apply(only_the_best).reset_index(drop=True)


,user_id,title,movie_id,rating
0,1,Hercules (1997),1566,1
1,1,Erin Brockovich (2000),3408,1
2,1,Toy Story (1995),1,1
3,1,Titanic (1997),1721,1
4,1,One Flew Over the Cuckoo's Nest (1975),1193,1
...,...,...,...,...
800162,6040,"Paris, Texas (1984)",1305,1
800163,6040,Breaking Away (1979),3359,1
800164,6040,"Simple Plan, A (1998)",2391,1
800165,6040,"Haunted World of Edward D. Wood Jr., The (1995)",722,0


In [51]:
from sklearn.model_selection import train_test_split

train, val = train_test_split(df, test_size=0.2, random_state=7)

train = train.copy()
val = val.copy()

y_train = train['rating']
train.drop(['rating'], axis=1, inplace=True)

train['user_id'] = train['user_id'].astype('str')
train['movie_id'] = train['movie_id'].astype('str')

y_val = val["rating"]
val.drop(['rating'], axis=1, inplace=True)
val['user_id'] = val['user_id'].astype('str')
val['movie_id'] = val['movie_id'].astype('str')

## Utility

In [52]:
def evaluate(predict_f,data_train,data_test):
    """ RMSE-based predictive performance evaluation with pandas. """
    ids_to_estimate = zip(data_test.user_id, data_test.movie_id)
    list_users = set(data_train.user_id)
    estimated = np.array([predict_f(u,i) if u in list_users else 3 for (u,i) in ids_to_estimate ])
    real = data_test.rating.values
    return compute_rmse(estimated, real)

def compute_rmse(y_pred, y_true):
    """ Compute Root Mean Squared Error. """
    return np.sqrt(np.mean(np.power(y_pred - y_true, 2)))

def precision(recommended_items, relevant_items):
    is_relevant = np.in1d(recommended_items, relevant_items, assume_unique=True)
    precision_score = np.sum(is_relevant, dtype=np.float32) / len(is_relevant)
    
    return precision_score

def recall(recommended_items, relevant_items):  
    is_relevant = np.in1d(recommended_items, relevant_items, assume_unique=True)
    recall_score = np.sum(is_relevant, dtype=np.float32) / relevant_items.shape[0]
    
    return recall_score

def AP(recommended_items, relevant_items):
   
    is_relevant = np.in1d(recommended_items, relevant_items, assume_unique=True)
    # Cumulative sum: precision at 1, at 2, at 3 ...
    p_at_k = is_relevant * np.cumsum(is_relevant, dtype=np.float32) / (1 + np.arange(is_relevant.shape[0]))
    ap_score = np.sum(p_at_k) / np.min([relevant_items.shape[0], is_relevant.shape[0]])

    return ap_score

def evaluate_algorithm_top(test, recommender_object, at=25, thr_relevant = 0.85):
    
    cumulative_precision = 0.0
    cumulative_recall = 0.0
    cumulative_AP = 0.0
    
    num_eval = 0


    for user_id in tqdm(test.user_id.unique()):
        
        relevant_items = test[test.user_id==user_id]
        thr = np.quantile(relevant_items.rating,thr_relevant)
        relevant_items = np.array(relevant_items[relevant_items.rating >=thr].movie_id.values)
        if len(relevant_items)>0:
            
            recommended_items = recommender_object.predict_top(user_id, at=at)
            num_eval+=1

            cumulative_precision += precision(recommended_items, relevant_items)
            cumulative_recall += recall(recommended_items, relevant_items)
            cumulative_AP += AP(recommended_items, relevant_items)
            
    cumulative_precision /= num_eval
    cumulative_recall /= num_eval
    MAP = cumulative_AP / num_eval
    
    print("Recommender results are: Precision = {:.4f}, Recall = {:.4f}, MAP = {:.4f}".format(
        cumulative_precision, cumulative_recall, MAP)) 

## Attempt

In [77]:
import time
from sklearn.metrics import mean_squared_error

class FM():
    def __init__(self,k = 40,learning_rate = 10,solver='stochastic',iterations = 100,regularizer = 0.00035,verbose = True):
        self.learning_rate = learning_rate
        self.solver=solver
        self.k = k
        self.iterations = iterations
        self.indices = {}
        self.regularizer = regularizer
        self.verbose = verbose
        
        
    def set_indices(self,X):
        i = 0
        cols = {}
        for col in X.columns:
            cols[col] = i
            i += 1

        indices = {}
        nz = 0
        for col in X.columns:
            if X[col].dtype == 'object':
                indices[cols[col]] = {}
                colset = set(X[col])
                for a in colset:
                    indices[cols[col]][a] = nz
                    nz += 1
            else:
                indices[cols[col]] = nz
                nz += 1
        self.indices = indices
        self.nz = nz
        
    
    def getIndexVal(self, col, val):
        if isinstance(self.indices[col], int) or isinstance(self.indices[col], float):
            return self.indices[col], val
        else:
            try:
                return self.indices[col][val], 1.
            except:
                #print(col, val)
                return 0., 0.
            
    def getIndexValArray(self, arr):
        out = []
        for i in range(len(arr)):
            out.append(self.getIndexVal(i, arr[i]))
        return out
    
    def initialize_params(self, X, y):
        self.set_indices(X)
        self.w0 = np.mean(y)
        self.w1 = np.zeros(self.nz)
        self.v = np.random.normal(scale=0.1,size=(self.nz, self.k))
        
    def sgd(self, X, y, verbose = True):

        learning_rate = self.learning_rate
        regularizer = self.regularizer
        w0 = self.w0
        w1 = self.w1
        v = self.v
        nz = self.nz

        m = X.shape[0]
        n = X.shape[1]
        
        #SGD
        for epoch in range(self.iterations):
            start_time = time.perf_counter()

            preds = []
            for i in range(m):

                _x = X[i, :]
                _ivals = self.getIndexValArray(_x)
                ind = {}
                sum1 = 0
                sum2 = 0
                sum3 = np.zeros(self.k)

                for col in range(n):
                    index, val = _ivals[col]
                    ind[index] = val
                    sum1 += w1[index] * val

                for f in range(self.k):
                    s1 = 0.0
                    s2 = 0.0
                    for col in range(n):
                        index, val = _ivals[col]
                        temp = v[index, f] * val
                        s1 += temp
                        s2 += temp*temp
                    sum3[f] = s1
                    sum2 += s1*s1 - s2

                y_hat = w0 + sum1 + 0.5*sum2
                y_hat = max(1., y_hat)
                y_hat = min(5., y_hat)
                res = (y_hat - y[i])
                if self.verbose:
                    preds.append(abs(res)**2)

                b = learning_rate*regularizer
                # update rule for w0
                w0 = w0 - learning_rate * res - learning_rate * w0 * regularizer

                for col in range(n):
                    #index = col
                    #if col in ind:
                    #    val = ind[col]
                    index, val = _ivals[col]
                    temp = learning_rate * val * res
                    w1[index] -= (temp + b*w1[index])
                    for f in range(self.k):
                        v[index, f] -= (temp * (sum3[f] - v[index, f] * val) + b*v[index, f])
                    #else:
                    #    w1[index] -= b*w1[index]
                    #    for f in range(self.k):
                    #        v[index, f] -= b*v[index, f]
            # val_y_pred = self.predict(self.val_X)
            # val_mse = mean_squared_error(val_y_pred, self.val_y)
            print("epoch {} time {} mse {}".format(epoch, time.perf_counter()-start_time, np.mean(preds)))
            # print("val_mse {}".format(val_mse))
        self.w0 = w0
        self.w1 = w1
        self.v = v
    
    
    def fit(self,X,y,val_X, val_y):
        self.initialize_params(X, y)
        X = np.array(X)
        y = np.array(y)
        self.val_X = val_X
        self.val_y = val_y
        self.sgd(X, y)
        return
    
    def predict(self, X, verbose=True):

        X = np.array(X)
        
        w0 = self.w0
        w1 = self.w1
        v = self.v
        nz = self.nz

        
        m = X.shape[0]
        n = X.shape[1]

        preds = []
        for i in range(m):


            _x = X[i, :]
            _ivals = self.getIndexValArray(_x)
            ind = {}
            sum1 = 0
            sum2 = 0
            sum3 = np.zeros(self.k)

            for col in range(n):

                index, val = _ivals[col]
                ind[index] = val

                index = int(index)

                sum1 += w1[index] * val


            for f in range(self.k):
                s1 = 0
                s2 = 0
                for col in range(n):
                    index, val = _ivals[col]
                    index = int(index)
                    temp = v[index, f] * val
                    s1 += temp
                    s2 += temp*temp
                sum3[f] = s1
                s1 = s1*s1
                sum2 += s1 - s2

            y_hat = w0 + sum1 + 0.5*sum2
            y_hat = max(1., y_hat)
            y_hat = min(5., y_hat)
            preds.append(y_hat)

        return preds

    def predict_top(self, user_id, at=25, remove_seen=True, return_titles = False):
        '''Given a user_id predict its top AT items'''
        seen_items = self.train[self.train.user_id==user_id].movie_id.values
        unseen_items = set(self.train.movie_id.values) - set(seen_items)
        predictions = [(item_id,self.predict_score(user_id,item_id)) for item_id in unseen_items]

        sorted_predictions = sorted(predictions, key=lambda x: x[1],reverse = True)[:at]
        if(return_titles):
            return [self.movie_id2title[i[0]] for i in sorted_predictions]
        return [i[0] for i in sorted_predictions]

In [78]:
import time
from sklearn.metrics import mean_squared_error

class FM2():
    def __init__(self,k = 40,learning_rate = 10,solver='stochastic',iterations = 100,regularizer = 0.00035,verbose = True):
        self.learning_rate = learning_rate
        self.solver=solver
        self.k = k
        self.iterations = iterations
        self.indices = {}
        self.regularizer = regularizer
        self.verbose = verbose
        
        
    def set_indices(self,X):
        i = 0
        cols = {}
        for col in X.columns:
            cols[col] = i
            i += 1

        indices = {}
        nz = 0
        for col in X.columns:
            if X[col].dtype == 'object':
                indices[cols[col]] = {}
                colset = set(X[col])
                for a in colset:
                    indices[cols[col]][a] = nz
                    nz += 1
            else:
                indices[cols[col]] = nz
                nz += 1
        self.indices = indices
        self.nz = nz
        
    
    def getIndexVal(self, col, val):
        if isinstance(self.indices[col], int) or isinstance(self.indices[col], float):
            return self.indices[col], val
        else:
            try:
                return self.indices[col][val], 1.
            except:
                #print(col, val)
                return 0., 0.
            
    def getIndexValArray(self, arr):
        out = []
        for i in range(len(arr)):
            out.append(self.getIndexVal(i, arr[i]))
        return out
    
    def initialize_params(self, X, y):
        self.set_indices(X)
        self.w0 = np.mean(y)
        self.w1 = np.zeros(self.nz)
        self.v = np.random.normal(scale=0.1,size=(self.nz, self.k))
        
    def sgd(self, X, y, verbose = True):

        learning_rate = self.learning_rate
        regularizer = self.regularizer
        w0 = self.w0
        w1 = self.w1
        v = self.v
        nz = self.nz

        m = X.shape[0]
        n = X.shape[1]
        
        #SGD
        for epoch in range(self.iterations):
            start_time = time.perf_counter()

            preds = []
            for i in range(m):

                _x = X[i, :]
                _ivals = self.getIndexValArray(_x)
                ind = {}
                sum1 = 0
                sum2 = 0
                sum3 = np.zeros(self.k)

                for col in range(n):
                    index, val = _ivals[col]
                    ind[index] = val
                    sum1 += w1[index] * val

                for f in range(self.k):
                    s1 = 0.0
                    s2 = 0.0
                    for col in range(n):
                        index, val = _ivals[col]
                        temp = v[index, f] * val
                        s1 += temp
                        s2 += temp*temp
                    sum3[f] = s1
                    sum2 += s1*s1 - s2

                y_hat = w0 + sum1 + 0.5*sum2
                y_hat = max(0., y_hat)
                y_hat = min(1., y_hat)
                res = (y_hat - y[i])
                if self.verbose:
                    preds.append(abs(res)**2)

                b = learning_rate*regularizer
                # update rule for w0
                w0 = w0 - learning_rate * res - learning_rate * w0 * regularizer

                for col in range(n):
                    #index = col
                    #if col in ind:
                    #    val = ind[col]
                    index, val = _ivals[col]
                    temp = learning_rate * val * res
                    w1[index] -= (temp + b*w1[index])
                    for f in range(self.k):
                        v[index, f] -= (temp * (sum3[f] - v[index, f] * val) + b*v[index, f])
                    #else:
                    #    w1[index] -= b*w1[index]
                    #    for f in range(self.k):
                    #        v[index, f] -= b*v[index, f]
            # val_y_pred = self.predict(self.val_X)
            # val_mse = mean_squared_error(val_y_pred, self.val_y)
            print("epoch {} time {} mse {}".format(epoch, time.perf_counter()-start_time, np.mean(preds)))
            # print("val_mse {}".format(val_mse))
        self.w0 = w0
        self.w1 = w1
        self.v = v
    
    
    def fit(self,X,y,val_X, val_y):
        self.initialize_params(X, y)
        X = np.array(X)
        y = np.array(y)
        self.val_X = val_X
        self.val_y = val_y
        self.sgd(X, y)
        return
    
    def predict(self, X, verbose=True):

        X = np.array(X)
        
        w0 = self.w0
        w1 = self.w1
        v = self.v
        nz = self.nz

        
        m = X.shape[0]
        n = X.shape[1]

        preds = []
        for i in range(m):


            _x = X[i, :]
            _ivals = self.getIndexValArray(_x)
            ind = {}
            sum1 = 0
            sum2 = 0
            sum3 = np.zeros(self.k)

            for col in range(n):

                index, val = _ivals[col]
                ind[index] = val

                index = int(index)

                sum1 += w1[index] * val


            for f in range(self.k):
                s1 = 0
                s2 = 0
                for col in range(n):
                    index, val = _ivals[col]
                    index = int(index)
                    temp = v[index, f] * val
                    s1 += temp
                    s2 += temp*temp
                sum3[f] = s1
                s1 = s1*s1
                sum2 += s1 - s2

            y_hat = w0 + sum1 + 0.5*sum2
            y_hat = max(0., y_hat)
            y_hat = min(1., y_hat)
            preds.append(y_hat)

        return preds

    def predict_top(self, user_id, at=25, remove_seen=True, return_titles = False):
        '''Given a user_id predict its top AT items'''
        seen_items = self.train[self.train.user_id==user_id].movie_id.values
        unseen_items = set(self.train.movie_id.values) - set(seen_items)
        predictions = [(item_id,self.predict_score(user_id,item_id)) for item_id in unseen_items]

        sorted_predictions = sorted(predictions, key=lambda x: x[1],reverse = True)[:at]
        if(return_titles):
            return [self.movie_id2title[i[0]] for i in sorted_predictions]
        return [i[0] for i in sorted_predictions]

In [86]:
f = FM2(k=10, iterations = 30, learning_rate = 0.01, regularizer=0.03)
f.fit(X=train, y=y_train, val_X=val, val_y=y_val)

epoch 0 time 17.1722399580176 mse 0.19725081616280105
epoch 1 time 16.66016824997496 mse 0.18532614124266103
epoch 2 time 16.69018120795954 mse 0.18294891726401713
epoch 3 time 16.596537875011563 mse 0.18170667298644508
epoch 4 time 16.307179166993592 mse 0.18086663804929667
epoch 5 time 16.747880542010535 mse 0.1801984970045898
epoch 6 time 16.469946333032567 mse 0.17959966920219372
epoch 7 time 16.616040332999546 mse 0.17901114527003023
epoch 8 time 16.923137000005227 mse 0.17839096858686596
epoch 9 time 16.333649959007744 mse 0.17770447195995234
epoch 10 time 16.559552500024438 mse 0.17692290345718067
epoch 11 time 16.527180291013792 mse 0.176026867371397
epoch 12 time 16.72694116702769 mse 0.1750140246267897
epoch 13 time 16.334408207971137 mse 0.17390307900474783
epoch 14 time 16.61104929097928 mse 0.17272960162717238
epoch 15 time 16.68842283298727 mse 0.17153211368458798
epoch 16 time 16.376388749980833 mse 0.17034195572174968
epoch 17 time 16.636651249951683 mse 0.1691772551158

In [87]:
from sklearn.metrics import mean_squared_error
y_pred = f.predict(val)
rmse = compute_rmse(y_pred, y_val)
mse = mean_squared_error(y_pred, y_val)

print("RMSE:", rmse)
print("MSE:", mse)

RMSE: 0.42520527025553695
MSE: 0.18079952185308423


In [88]:
y_pred

[0.18027390775880422,
 0.5793209446538576,
 0.4512330635157794,
 0.9971136771805889,
 0.9927257285949418,
 0.9920305870391475,
 0.5725321913545652,
 0.4426281479896615,
 0.7995001680441364,
 0.6515089090107471,
 0.6907856573620962,
 0.7360195698719851,
 0.816751595968734,
 0.40456607445647846,
 0.13662107973988846,
 0.4844584228053082,
 0.7133174983818873,
 1.0,
 0.3977492139843588,
 0.3961112395866728,
 0.7612105310273278,
 0.16707576594812187,
 0.3523852151797844,
 0.6930651935229831,
 1.0,
 0.45175185866311013,
 0.6986217114795436,
 0.9168770968974347,
 0.6617094469542847,
 0.5716841825823332,
 1.0,
 0.5331630398837045,
 0.764043364123032,
 0.48265369000762065,
 0.324921625474917,
 0.9050351455563869,
 0.35289463985224845,
 0.6462308623538369,
 0.2228222306930103,
 0.25450375651734464,
 0.9650099267188638,
 0.5727334323768061,
 0.0,
 0.8248204124394506,
 1.0,
 0.8510429061984444,
 0.7969497422189235,
 0.3747933537514979,
 0.4747523826358122,
 0.6333854920041821,
 0.22098256820292939

Stanni-FM (60ite): 0.80 (MSE)

In [23]:
#f.predict_top(1, df, at=25)

TypeError: 'int' object is not subscriptable